In [1]:
import os  # for accessing the file system
from itertools import product  # for efficient looping process

import numpy as np  # for linear algebra
import pandas as pd  # for data analysis
# for image processing
import PIL
# for deep learning process
import tensorflow as tf
from keras.preprocessing import image
from keras.utils import image_dataset_from_directory
from PIL import Image
from tensorflow import keras

# Find the count of the files in the datasets directory


In [2]:
set_list = ["test_set", "training_set"]
cats_dogs = ["cats", "dogs"]
folderpath = [
    os.path.abspath(f"datasets/{sets}/{animals}")
    for sets, animals in product(set_list, cats_dogs)
]

count = sum(len(os.listdir(folderpath[i])) for i in range(len(folderpath)))
print(f"Total images found: {count} files belonging")


Total images found: 10028 files belonging


# Preparing all variables needed

In [12]:
size = 260
epoch_amounts = 50
labels = "inferred"
label_mode = "categorical"
interpolation = "nearest"

# Preparing the data
Before feeding the data to the model, we need to convert the images files to a float so the computer won't break the system.

In [11]:
def convert_to_float(img, label):
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img, label

In [14]:
train_ds = image_dataset_from_directory(
    "datasets/training_set",
    seed=1,
    labels=labels,
    label_mode=label_mode,
    color_mode="rgb",
    batch_size=32,
    image_size=(260, 260),
    interpolation=interpolation,
    shuffle=True,
)
test_ds = image_dataset_from_directory(
    "datasets/test_set",
    seed=1,
    labels=labels,
    label_mode=label_mode,
    color_mode="rgb",
    batch_size=32,
    image_size=(260, 260),
    interpolation=interpolation,
    shuffle=True,
)
print(train_ds, test_ds)


Found 8005 files belonging to 2 classes.
Found 2023 files belonging to 2 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 260, 260, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))> <BatchDataset element_spec=(TensorSpec(shape=(None, 260, 260, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>


In [16]:
# cats = list(train_ds.take(1))[0][0][0]
# Image.open(cats)

# Create the model using the Sequential API


## Assumptions

Now we have training data and train data. In test data, we have 8005 images.
And in the test data, we have 2023 images. We have 2 classes, which are cats and dogs.
I assume that the total images available is enough for training and validating the model.

Now let's jump into creating the model


In [17]:
model = keras.Sequential([
    # Input layer
    keras.layers.InputLayer(input_shape=(size, size, 3)),

    keras.layers.BatchNormalization(renorm=True),
    keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(),
    
    keras.layers.BatchNormalization(renorm=True),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(),

    keras.layers.BatchNormalization(renorm=True),
    keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(),

    keras.layers.BatchNormalization(renorm=True),
    keras.layers.Flatten(),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(2, activation='sigmoid')
])

In [18]:
adam_opt = keras.optimizers.Adam(epsilon=0.01)
model.compile(
    optimizer=adam_opt,
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
)

In [19]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=50,
    shuffle=True
)

Epoch 1/50
251/251 [==============================] - 667s 3s/step - loss: 2.4733 - categorical_accuracy: 0.5429 - val_loss: 0.6751 - val_categorical_accuracy: 0.6134
Epoch 2/50
251/251 [==============================] - 723s 3s/step - loss: 0.7998 - categorical_accuracy: 0.5634 - val_loss: 0.6787 - val_categorical_accuracy: 0.6169
Epoch 3/50
251/251 [==============================] - 788s 3s/step - loss: 0.6851 - categorical_accuracy: 0.6057 - val_loss: 0.7101 - val_categorical_accuracy: 0.6214
Epoch 4/50
251/251 [==============================] - 788s 3s/step - loss: 0.6015 - categorical_accuracy: 0.6598 - val_loss: 0.6282 - val_categorical_accuracy: 0.6535
Epoch 5/50
251/251 [==============================] - 789s 3s/step - loss: 0.5607 - categorical_accuracy: 0.6962 - val_loss: 0.6043 - val_categorical_accuracy: 0.6634
Epoch 6/50
251/251 [==============================] - 675s 3s/step - loss: 0.5252 - categorical_accuracy: 0.7187 - val_loss: 0.5835 - val_categorical_accuracy: 0.676

In [21]:
model.save('my_model')

INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets
